In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [2]:
def read_krx_code():
    """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
        'download&searchType=13'
    krx = pd.read_html(url, header=0)[0]
    krx = krx[['종목코드', '회사명']]
    krx = krx.rename(columns={'종목코드' : 'code', '회사명' : 'company'})
    krx.code = krx.code.map('{:06d}'.format)
    
    return krx

In [3]:
def getIS(stock_code, rpt_type, freq):
    """[FnGuide] 공시기업의 최근 3개 연간 및 4개 분기 손익계산서를 수집하는 함수
    
    Parameters
    ==========
    stock_code : str, 종목코드
    rpt_type   : str, 재무제표 종류
        'Consolidated'(연결), 'Unconsolidated'(별도)
    freq       : str, 연간 및 분기보고서
        'A'(연간), 'Q'(분기)
    """
    items_en = ['rev', 'cgs', 'gross', 'sga', 'sga1', 'sga2', 'sga3', 'sga4', 'sga5', 'sga6', 'sga7', 'sga8', 'opr', 'opr_', # 14
                'fininc', 'fininc1', 'fininc2', 'fininc3', 'fininc4', 'fininc5', # 6
                'fininc6', 'fininc7', 'fininc8', 'fininc9', 'fininc10', 'fininc11', # 6
                'fincost', 'fincost1', 'fincost2', 'fincost3', 'fincost4', 'fincost5', # 6
                'fincost6', 'fincost7', 'fincost8', 'fincost9', 'fincost10', # 5
                'otherrev', 'otherrev1', 'otherrev2', 'otherrev3', 'otherrev4', 'otherrev5', 'otherrev6', 'otherrev7', 'otherrev8', # 9
                'otherrev9', 'otherrev10', 'otherrev11', 'otherrev12', 'otherrev13', 'otherrev14', 'otherrev15', 'otherrev16', # 8
                'othercost', 'othercost1', 'othercost2', 'othercost3', 'othercost4', 'othercost5', # 6
                'othercost6', 'othercost7', 'othercost8', 'othercost9', 'othercost10', 'othercost11', 'othercost12', # 7
                'otherpl', 'otherpl1', 'otherpl2', 'otherpl3', 'otherpl4', 'ebit', 'tax', 'contop', 'discontop', 'netinc'] # 12
    
    if rpt_type.upper() == 'CONSOLIDATED':
        # 연결 연간 손익계싼서(ReportGB=D)
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?" + "pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701".format(stock_code)
        items_en = items_en + ['netinc1', 'netinc2']
        
    else:
        # 별도 연간 손익계산서(ReportGB=B)
        url = "https://comp.fnguide.com/SV02/asp/SVD_Finance.asp?" + "pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701".format(stock_code)
        
    # Header 설정
    headers = {
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
    }
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    if freq.upper() == 'A': # 연간 손익계산서 영역 추출
        is_a = soup.find(id = 'divSonikY')
        num_col = 4 # 최근 4개 연간 데이터
    else: # 분기 손익계산서 영역 추출 freq.upper() == 'Q'
        is_a = soup.find(id = 'divSonikQ')
        num_col = 4 # 최근 4개 분기 데이터
        
    # 존재하지 않는 데이터(None) 처리
    if is_a is None:
        return
    else:
        is_a = is_a.find_all(['tr'])
    
    # 연간 손익계산서 항목 펼친 뒤 하위 항목 추출
    items_kr = [is_a[m].find(['th']).get_text().replace('\n', '').replace('\xa0', '').replace('계산에 참여한 계정 펼치기', '')
                for m in range(1, len(is_a))]
    
    # 최근 3개 연간 손익계산서 자료 수집
    period = [is_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
    
    # 항목별 최근 3개 연간데이터 불러오기
    for item, i in zip(items_en, range(1, len(is_a))):
        temps = []
        for j in range(0, num_col):
            temp = [float(is_a[i].find_all('td')[j]['title'].replace(',', '').replace('\xa0', '')) \
                    if is_a[i].find_all('td')[j]['title'].replace(',', '').replace('\xa0', '') != '' \
                    else (0 if is_a[i].find_all('td')[j]['title'].replace(',', '').replace('\xa0', '') == '-0' \
                         else 0)]
            temps.append(temp[0])
            
        # item_en 내 각 항목을 global 변수로 지정하고 값 저장
        globals()[item] = temps
        
    # 지배/비지배 항목 청리
    if rpt_type.upper() == 'CONSOLIDATED': # 연결 연간 손익계산서는 아무 것도 하지 않음
        pass
    
    else: # 별도 연간 손익계산서 해당 항목을 Null값으로 채움
        globals()['netinc1'], globals()['netinc2'] = [np.NaN] * num_col, [np.NaN] * num_col
        
    # 손익계산서 각 컬럼에 위에서 저장한 global 변수값을 지정하여 dataframe으로 변환
    is_domestic = pd.DataFrame({'종목' : stock_code, '기간' : period,
                                '매출액' : rev, '매출원가' : cgs, '매출총이익' : gross,
                                '판매비와관리비' : sga, '영업이익(발표기준)' : opr_,
                                '금융수익' : fininc, '금융원가' : fincost,
                                '기타수익' : otherrev, '기타비용' : othercost,
                                '관계기업관련손익' : otherpl, 'EBIT' : ebit,
                                '법인세비용' : tax, '계속영업이익': contop,
                                '중단영업이익' : discontop, '당기순이익' : netinc,
                                '지배주주순이익' : globals()['netinc1'],
                                '비지배주주순이익' : globals()['netinc2']})
    
    is_domestic['재무제표종류'] = rpt_type + '_' + freq.upper()
    
    return is_domestic

In [4]:
import pymysql # python에서 mysql을 사용하는 패키지

import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

In [5]:
def connectMySQL():
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = 'js0815' # 개인 password
    db = '16010980_my1st_db' # DB 이름
    
    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user, password, server, db))
    
    return engine

In [6]:
def setIS_toDB(df, name):
    engine = connectMySQL()
    
    df.to_sql(name=name, con=engine, if_exists='append', index=False,
        dtype={
            '종목' : sqlalchemy.types.VARCHAR(10),
            '기간' : sqlalchemy.types.VARCHAR(10),
            '매출액' : sqlalchemy.types.Float(),
            '매출원가' : sqlalchemy.types.Float(),
            '매출총이익' : sqlalchemy.types.Float(),
            '판매비와관리비' : sqlalchemy.types.Float(),
            '영업이익(발표기준)' : sqlalchemy.types.Float(),
            '금융수익' : sqlalchemy.types.Float(),
            '금융원가' : sqlalchemy.types.Float(),
            '기타수익' : sqlalchemy.types.Float(),
            '기타비용' : sqlalchemy.types.Float(),
            '관계기업관련손익' : sqlalchemy.types.Float(),
            'EBIT' : sqlalchemy.types.Float(),
            '법인세비용' : sqlalchemy.types.Float(),
            '계속영업이익' : sqlalchemy.types.Float(),
            '중단영업이익' : sqlalchemy.types.Float(),
            '당기순이익' : sqlalchemy.types.Float(),
            '지배주주순이익' : sqlalchemy.types.Float(),
            '비지배주주순이익' : sqlalchemy.types.Float()
        }
    )
    
    # DB 접속 해제
    engine.dispose()
    # cur.close()
    # conn.close()

In [8]:
from tqdm import tqdm_notebook

krx_df = read_krx_code()

code_list = krx_df['code'].values.tolist()

df = pd.DataFrame()
for code in tqdm_notebook(code_list):
    df = df.append(getIS(code, 'Consolidated', 'A'))
    
print("데이터베이스에 연간 손익계산서 저장 시작!")
setIS_toDB(df, 'Income Statement')
print("데이터베이스에 연간 손익계산서 저장 완료!")

df = pd.DataFrame()
for code in tqdm_notebook(code_list):
    df = df.append(getIS(code, 'Consolidated', 'Q'))
    
print("데이터베이스에 분기 손익계산서 저장 시작!")
setIS_toDB(df, 'Income Statement')
print("데이터베이스에 분기 손익계산서 저장 완료!")

C:\Users\User\AppData\Local\Temp\ipykernel_37252\3545073925.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for code in tqdm_notebook(code_list):


  0%|          | 0/2500 [00:00<?, ?it/s]

데이터베이스에 연간 손익계산서 저장 시작!
데이터베이스에 연간 손익계산서 저장 완료!


D:\anaconda3\lib\site-packages\pandas\io\sql.py:1685: UserWarning: The provided table name 'Income Statement' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\Users\User\AppData\Local\Temp\ipykernel_37252\3545073925.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for code in tqdm_notebook(code_list):


  0%|          | 0/2500 [00:00<?, ?it/s]

데이터베이스에 분기 손익계산서 저장 시작!
데이터베이스에 분기 손익계산서 저장 완료!
